In [6]:
import os
import json
from PIL import Image
import numpy as np
import random
import time


# 裁剪并调整图像大小的函数
def load_512(image_path: str, left=0, right=0, top=0, bottom=0):
    # 打开图像并转换为 numpy 数组，保留 RGB 通道
    image = np.array(Image.open(image_path))[:, :, :3]
    h, w, c = image.shape

    # 对裁剪边界进行修正，避免超出图像尺寸
    left = min(left, w - 1)
    right = min(right, w - left - 1)
    top = min(top, h - left - 1)
    bottom = min(bottom, h - top - 1)

    # 根据给定的边界裁剪图像
    image = image[top : h - bottom, left : w - right]

    h, w, c = image.shape

    # 如果图像高度小于宽度，裁剪或填充使其成为正方形
    if h < w:
        offset = (w - h) // 2
        image = image[:, offset : offset + h]
    elif w < h:
        offset = (h - w) // 2
        image = image[offset : offset + w]

    # 使用 PIL 将图像调整为 512x512
    image = np.array(Image.fromarray(image).resize((512, 512)))
    return image


# 读取 `src` 目录中的所有图像，裁剪并保存到 `images` 目录
def process_and_save_images(src_dir: str, dest_dir: str):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)  # 创建 `images` 目录（如果不存在）

    current_time = time.localtime()
    hour_minute = f"{current_time.tm_hour:02d}{current_time.tm_min:02d}"

    # 获取 src 目录中的所有图片文件
    image_files = [
        f for f in os.listdir(src_dir) if f.lower().endswith(("png", "jpeg", "jpg"))
    ]

    # 用于存储 JSON 数据
    image_data = {"images": []}

    # 处理每个图像文件
    for index, image_file in enumerate(image_files):
        image_path = os.path.join(src_dir, image_file)
        # 裁剪图像
        image = load_512(image_path)
        
        dest_filename = f"{hour_minute}-{index + 1:02d}.png"
        dest_path = os.path.join(dest_dir, dest_filename)

        # 保存裁剪后的图像
        Image.fromarray(image).save(dest_path)
        print(f"已保存: {dest_path}")

        # 添加到 JSON 数据
        image_data["images"].append(
            {
                "path": dest_path,
                "source_caption": f"a photo of a cat",
                "target_caption": f"a photo of a tiger",
            }
        )

    # 将数据写入 JSON 文件
    with open(os.path.join(dest_dir, "data.json"), "w", encoding="utf-8") as json_file:
        json.dump(image_data, json_file, ensure_ascii=False, indent=4)
    print(f"已保存 JSON 文件：{os.path.join(dest_dir, 'data.json')}")


# 调用示例
process_and_save_images(
    "src", "images"
)  # 假设图像文件存储在 `src` 目录，裁剪后保存到 `images` 目录

已保存: images/2100-01.png
已保存: images/2100-02.png
已保存: images/2100-03.png
已保存: images/2100-04.png
已保存: images/2100-05.png
已保存 JSON 文件：images/data.json
